<a href="https://colab.research.google.com/github/mcroning/PRProp3D/blob/main/video_writer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook to priduce and save videos for large COLAB runs where runtime is disconnected during  normal operation of PRCoupler. In this case run PRcoupler_large_videos to save necessary files to temporary google drive. Then run this notebook to generate and save the movies to the Google folder specified when PRcoupler_large_videos was run. This notebook should be run in a large memory CPU instance of COLAB


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path

from matplotlib import rcParams
import matplotlib.animation as animation
from matplotlib.patches import Circle

import matplotlib.animation as animation
from IPython import display as ipydisplay

In [2]:
from google.colab import drive
drive.mount('/content/drive')
pathf_temp='/content/drive/MyDrive/temp_files/'
if not Path(pathf_temp).is_dir():
  Path.mkdir(Path(pathf_temp))

In [3]:
with open(pathf_temp+'dict_movie.json') as f:
  prdata=json.load(f)
npzarrays=np.load(pathf_temp+'movie_data.npz')
ixz=npzarrays['ixz']
output_direction_cosines=npzarrays['output_direction_cosines']
imout_cpu=npzarrays['imout_cpu']

sx=npzarrays['sx']
wdisp1=prdata['wdisp1']
wdisp4=prdata['wdisp4']
wdisp5=prdata['wdisp5']
extent_crop=prdata['extent_crop']
extent_s=prdata['extent_s']
xaper=prdata['xaper']
yaper=prdata['yaper']
xsamp=int(prdata['xsamp'])
ysamp=int(prdata['ysamp'])
thout1=prdata['thout1']
tsteps=prdata['tsteps']
jt_last_good=prdata['jt_last_good']
gl=prdata['gl']
abort=prdata['abort']
pathf=prdata['pathf']
fanning_study=prdata['fanning_study']

In [ ]:
# @title Generate movies for time dependent calculations

#load prdata
#
plt.rcParams['animation.embed_limit'] = 40

  #if levelsOK.value: #old code to check image level values

ims=[]
fig4 = plt.figure()
ax10=fig4.add_subplot()
ax10.set_xlabel('z mm')
ax10.set_ylabel('x mm')
ax10.set_title( r'cross section  $\gamma$l '+str(prdata['gl']))  # normal
if abort:
  video_steps=jt_last_good//4
else:
  video_steps=tsteps//4
for i in range(video_steps):
  im=ax10.imshow(ixz[i].T,
        extent=[prdata['rlen']/1000,0,-xaper/2000,xaper/2000],
        vmax=wdisp5)#,cmap='terrain')
  ims.append([im])
cbar=fig4.colorbar(im,ax=ax10,location='bottom')
ani1 = animation.ArtistAnimation(fig4, ims, interval=100, blit=True,
                              repeat_delay=1000)  #fig4
video=ani1.to_jshtml()
html=ipydisplay.HTML(video)
ipydisplay.display(html)
plt.close(fig4)


ims=[]
fig5 = plt.figure(figsize=(5, 5), dpi=100)

ax11=fig5.add_subplot()
ax11.set_xlabel(r'x ($\mu$m)')
ax11.set_ylabel(r'y ($\mu$m)')
ax11.set_title( r'output image $\gamma \ell$ '+str(prdata['gl']))
ymax=yaper/1000/2/2
# in mm 1st /2 is for half y aperture spanning,
#2nd /2 is for zero centered yaper

#for i in range(np.shape(imout_cpu)[0]):
for i in range(video_steps):

  im=ax11.imshow(abs(np.rot90(imout_cpu[i],k=3)),vmax=wdisp1,
  extent=extent_crop)
  ims.append([im])
cbar=fig5.colorbar(im,ax=ax11)
ani2 = animation.ArtistAnimation(fig5, ims, interval=100, blit=True,
                              repeat_delay=1000)
video=ani2.to_jshtml()
html=ipydisplay.HTML(video)
ipydisplay.display(html)
plt.close(fig5)



#      ffout,extent_ff=fx_to_angular(ift_cpu[-1])
#      ffout=ffout.T/np.sum(abs(amp.get())**2)*dx*dy

#for i in range(len(iftg)):
if fanning_study:
  #ift2=ift_cpu/np.sum(abs(ampout[jt_last_good//4]).get()**2)*dx*dy
  #ift3,ex3 = fx_to_angular(ift2)
  #ift3=ffout.T/np.sum(abs(amp.get())**2)*dx*dy
  fig6 = plt.figure(figsize=(7, 5), dpi=100)
  ax12=fig6.add_subplot()
  ax12.set_xlabel( 'x propagation angle')
  ax12.set_ylabel( 'y propagation angle')
  ax12.set_title( r'far field $\gamma \ell$ '+str(prdata['gl']))
  patch=Circle((0,0),radius=1,color='w',fill=False)
  ax12.add_patch(patch)
  ims=[]



  #ift3=np.zeros((video_steps,xsamp,ysamp))

  for i in range(video_steps):

      im=ax12.imshow(output_direction_cosines[i].T,extent=extent_s,
                      vmax=wdisp4)
      ims.append([im])
  ani3 = animation.ArtistAnimation(fig6, ims, interval=100, blit=True,
                                  repeat_delay=1000)
  video=ani3.to_jshtml()
  html=ipydisplay.HTML(video)
  ipydisplay.display(html)
  plt.close(fig6)
  #if False:

  fig2a,ax5 = plt.subplots() #output far field
  ax5.set_xlabel( 'x direction cosine')
  ax5.set_ylabel('y direction  cosine')
  ax5.set_title( r'far field $\gamma \ell$ '+str(prdata['gl']))
  ims=[]

  iroll=np.argmin(abs(np.fft.fftshift(sx)+np.sin(thout1)))

  for i in range(video_steps):

    img=ax5.imshow(np.roll(output_direction_cosines[i].T,-iroll-xsamp//2,axis=1),extent=extent_s,
                      vmax=wdisp4)

    ax5.set_xlim([(-0.02),(min(0.72,extent_s[1]))])
    ylimit=min((0.25),(min(0.25,extent_s[3])))
    ax5.set_ylim([-ylimit,ylimit])
    ims.append([img])
  ani3a = animation.ArtistAnimation(fig2a, ims, interval=100, blit=True,
                                  repeat_delay=1000)
  video=ani3a.to_jshtml()
  html=ipydisplay.HTML(video)
  ipydisplay.display(html)
  plt.close(fig2a)

In [6]:
writervideo=animation.FFMpegWriter(fps=5)
ani1.save(filename=pathf+"/ixz.mp4")
ani2.save(filename=pathf+"/imout.mp4")
if fanning_study:
  ani3.save(filename=pathf+"/farfield.mp4")
  ani3a.save(filename=pathf+"/farfield_compact.mp4")